Importing data from kaggle

In [3]:
!pip install -q kaggle
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

import os
os.environ['KAGGLE_CONFIG_DIR'] = "/root/.kaggle/"

!ls /root/.kaggle/

import zipfile

!kaggle datasets download -d akash2907/bird-species-classification
with zipfile.ZipFile("bird-species-classification.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")


Saving kaggle.json to kaggle.json
kaggle.json
100% 1.37G/1.37G [01:02<00:00, 25.0MB/s]
100% 1.37G/1.37G [01:02<00:00, 23.5MB/s]


Training and model

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2)
test_gen = ImageDataGenerator(rescale=(1./255))

In [5]:
train = train_gen.flow_from_directory('/content/dataset/test_data/test_data',
                                      target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)
test = test_gen.flow_from_directory('/content/dataset/test_data/test_data',
                                    target_size=(120, 120),
                                      class_mode='categorical', 
                                      batch_size=8)

Found 157 images belonging to 16 classes.
Found 157 images belonging to 16 classes.


In [6]:
train.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [7]:
test.class_indices

{'blasti': 0,
 'bonegl': 1,
 'brhkyt': 2,
 'cbrtsh': 3,
 'cmnmyn': 4,
 'gretit': 5,
 'hilpig': 6,
 'himbul': 7,
 'himgri': 8,
 'hsparo': 9,
 'indvul': 10,
 'jglowl': 11,
 'lbicrw': 12,
 'mgprob': 13,
 'rebimg': 14,
 'wcrsrt': 15}

In [8]:
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Dense
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(Convolution2D(12, (3, 3), activation='relu', input_shape=(120, 120, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(24, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(36, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(62, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='softmax'))


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 118, 12)      336       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 59, 12)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 57, 24)        2616      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 28, 24)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 36)        7812      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 13, 36)       0

In [10]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(train,batch_size=8,validation_data=test,epochs=25)

Epoch 1/25
20/20 [==============================] - 109s 5s/step - loss: 2.7894 - accuracy: 0.0828 - val_loss: 2.7479 - val_accuracy: 0.1401
Epoch 2/25
20/20 [==============================] - 96s 5s/step - loss: 2.7258 - accuracy: 0.1529 - val_loss: 2.6833 - val_accuracy: 0.1720
Epoch 3/25
20/20 [==============================] - 95s 5s/step - loss: 2.6740 - accuracy: 0.1465 - val_loss: 2.6389 - val_accuracy: 0.1529
Epoch 4/25
20/20 [==============================] - 95s 5s/step - loss: 2.6372 - accuracy: 0.1783 - val_loss: 2.6178 - val_accuracy: 0.2102
Epoch 5/25
20/20 [==============================] - 130s 7s/step - loss: 2.5875 - accuracy: 0.1720 - val_loss: 2.7715 - val_accuracy: 0.0510
Epoch 6/25
20/20 [==============================] - 95s 5s/step - loss: 2.6906 - accuracy: 0.1210 - val_loss: 2.5649 - val_accuracy: 0.1783
Epoch 7/25
20/20 [==============================] - 130s 7s/step - loss: 2.4922 - accuracy: 0.1911 - val_loss: 2.4025 - val_accuracy: 0.2548
Epoch 8/25
20/20 

In [12]:
model.save('birdsWeights.h5')

Testing

In [13]:
# Testing
import numpy as np
from tensorflow.keras.preprocessing import image

In [15]:
img1 = image.load_img('/content/dataset/test_data/test_data/gretit/11776135285_ccf938fa2e_o.jpg',target_size=(120,120))
img1 = image.img_to_array(img1)
img1 = np.expand_dims(img1,axis=0)
pred = np.argmax(model.predict(img1))
print(pred)
output = ['blasti','bonegl','brhkyt','cbrtsh','cmnmyn','gretit','hilpig','himbul','himgri','hsparo','indvul','jglowl','lbicrw','mgprob','rebimg','wcrsrt']
print(output[pred])

1/1 [==============================] - 0s 195ms/step
6
hilpig
